All external code used in this project is labeled with a comments "START OF EXTERNAL CODE" and "END OF EXTERNAL CODE". It comes with a inline reference to the source of the code.


Student code start

# Import dataset to the database

In [1]:
from tqdm.auto import tqdm
import pandas as pd
from sqlalchemy import create_engine

/Users/dima/dev/cm3010-databases-and-advanced-data-techniques-midterm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Create a database connection:

In [2]:
engine = create_engine('mysql+mysqlconnector://root:@localhost:3306/snpedia_db')

## SNPs

SNP relation contains information from two files: snps.csv and rsnums.csv
load data from these files into a Pandas DataFrames:

In [3]:
snps_df = pd.read_csv('dataset/snps.csv', index_col=0)
rsnums_df = pd.read_csv('dataset/rsnums.csv', index_col=0)

/var/folders/v1/whtp_6_j3p3gwb_dgdz6gkpw0000gn/T/ipykernel_13212/4195730567.py:2: DtypeWarning: Columns (17,19,20,29,31,32,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  rsnums_df = pd.read_csv('dataset/rsnums.csv', index_col=0)


Merge these DataFrames into a single DataFrame:

In [4]:
df = pd.merge(snps_df, rsnums_df, how="left", left_index=True, right_index=True)

Drop unused columns, rename columns to match the database schema:

In [5]:
df = df.drop(columns=["gene"]).rename(columns={"Description": "description", "Gene": "gene", "Chromosome": "chromosome", "Position": "position"})[["description", "chromosome", "gene", "position"]]

Write the DataFrame to the database:


In [6]:
df.to_sql("SNP", con=engine, if_exists='append', index_label="id")

111725

## Genotypes

Read genotypes from the file:

In [7]:
df = pd.read_csv('dataset/genotypes.csv', index_col=0)

Inner join with SNPs to get rid of the records with unknown SNPs (for some reason, there are a few of them):

In [8]:
df.rename(columns={"description": "genotype_description"}, inplace=True)

In [9]:
df = pd.merge(df, snps_df, how="inner", left_index=True, right_index=True)

Coy index to the column snp_id (foreign key):

In [10]:
df["snp_id"] = df.index

Fix column names to match the database schema:

In [11]:
df = df[["snp_id", "allele1", "allele2", "magnitude", "repute", "summary", "genotype_description"]]

In [12]:
df.rename(columns={"genotype_description": "description"}, inplace=True)

Clean up the data:

In [13]:

# Remove rows without allele1 or snp_id columns as they are not useful
df = df[df['snp_id'].notna() & df['allele1'].notna()]

# Remove duplicates:
df.drop_duplicates(subset=["snp_id", "allele1", "allele2"], inplace=True)

Lowercase the repute column to match the database schema:

In [14]:
df["repute"] = df["repute"].str.lower()

Save the DataFrame to the database:

In [15]:
df.to_sql("Genotype", con=engine, if_exists='append', index=False)

104687

## Categories

Read categories from the file:

In [16]:
df = pd.read_csv('dataset/categories.csv')

First, let's construct a list of unique categories to populate the category table:

In [17]:
categories_df = pd.DataFrame(df["name"][df["name"].notna()].unique(), columns=["name"])

The primary key starts from 1, so we need to add 1 to the index:

In [18]:
categories_df.index += 1

And change the index to match the "id" column:

In [19]:
categories_df.index.name = "id"

The list of categories is ready:

In [20]:
categories_df

,name
id,
1,On chip 23andMe v1
2,On chip 23andMe v2
3,On chip 23andMe v3
4,On chip 23andMe v4
5,On chip 23andMe v5
6,On chip Affy GenomeWide 6
7,On chip Affy500k
8,On chip Ancestry v2
9,On chip HumanOmni1Quad


Import it to the database:

In [21]:
categories_df.to_sql("Category", con=engine, if_exists='append')

16

## SNP_Category

To establish a many-to-many relation between SNPs and Categories, we need to populate a junction table SNP_Category:

In [22]:
categories_df["category_id"] = categories_df.index

In [23]:
df = pd.merge(df, categories_df, how="inner", left_on="name", right_on="name")

In [24]:
df = df.rename(columns={"ID": "snp_id"}).drop(columns=["name"])

All records in the junction table must be unique, so we need to drop duplicates:

In [25]:
df.drop_duplicates(subset=["snp_id", "category_id"], inplace=True)

In [26]:
df.to_sql("SNP_Category", con=engine, if_exists='append', index=False)

363060

## Literature

Load literature from the file:

In [27]:
df = pd.read_csv('dataset/pmids.csv')

First, let's construct a list of unique literature to populate the literature table:

In [28]:
literature_df = df[["PMID", "Title"]].rename(columns={"Title": "title"})

Data cleaning:

In [29]:
# Remove literature without PMID
literature_df = literature_df[df['PMID'].notna()]

# Remove letters from PMID
literature_df["PMID"] = literature_df["PMID"].str.replace("PMC", "")

# Remove rows with non-numeric PMIDs
literature_df = literature_df[literature_df["PMID"].str.isnumeric()]

# Remove duplicates:
literature_df = literature_df.drop_duplicates(subset=["PMID"])

# Truncate title to 255 characters
literature_df["title"] = literature_df["title"].str.slice(0, 255)

Again, the primary key starts from 1, so we need to add 1 to the index:

In [30]:
literature_df.index += 1

And change the index to match the "id" column:

In [31]:
literature_df.index.name = "id"

In [32]:
literature_df.head()

,PMID,title
id,,
4,17099056,NaN
124,23900608,NaN
125,19404532,NaN
126,22784820,NaN
129,23418762,Hereditary hemochromatosis


Import it to the database:

In [33]:
literature_df.to_sql("Literature", con=engine, if_exists='append')

33533

## SNP_Literature

To establish a many-to-many relation between SNPs and Literature, we need to populate a junction table SNP_Literature:

In [34]:
literature_df["literature_id"] = literature_df.index

In [35]:
snp_literature_df = pd.merge(df[["ID", "PMID"]].rename(columns={"ID": "snp_id"}), literature_df, right_on="PMID", left_on="PMID").drop(columns=["title", "PMID"]).drop_duplicates(subset=["snp_id", "literature_id"])

Import it to the database:

In [36]:
snp_literature_df.to_sql("SNP_Literature", con=engine, if_exists='append', index=False)

92763

Student code ends